In [1]:
import cvxpy as cp
import numpy as np
import csv

In [2]:
#import the resource codes
def load_file_60d(filename, filename2):
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        data={}
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break
            if next_line[1]!='1':
                break
            data[next_line[2]] = 'UNK'
#             data.append({next_line[2]:'UNK'})
    
    with open(filename2) as file:
        reader = csv.reader(file)
        next(reader)
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break
            if next_line[2]!='1':
                break
            if next_line[3] not in data:
#                 print ([next_line[3]])
                data[next_line[3]] = 'UNK'
    return data  

In [3]:
#map resource code to resource type
def code_to_type(filename, data):
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break
            if next_line[2] in data:
                if data[next_line[2]]!='UNK':
                    break
                data[next_line[2]] = next_line[3]

In [4]:
#add the resource types together
def wind_solar_added(filename, data, wind, solar, percent_wind , percent_solar):
    wind_total = 20271
    solar_total = 1008
    with open(filename) as file:
            reader = csv.reader(file)
            next(reader)
            while True:
                try:
                    next_line = next(reader)
                except StopIteration:
                    break
#                 print(data[next_line[3]])
                if next_line[1] not in (wind and solar):
                    solar[next_line[1]] = 0
                    wind[next_line[1]] = 0
                    percent_wind[next_line[1]] = 0
                if (data[next_line[3]])=='WIND':
                    wind[next_line[1]] = (float(wind[next_line[1]]) + float(next_line[4]))
                    percent_wind[next_line[1]] = (float(wind[next_line[1]]) + float(next_line[4]))/wind_total
                if (data[next_line[3]])=='PVGR':
                    solar[next_line[1]]= float(solar[next_line[1]]) + float(next_line[4])
                    percent_solar[next_line[1]] = (float(solar[next_line[1]]) + float(next_line[4]))/solar_total
#     print (solar)

In [5]:
#get the total amount of solar and wind per region
def map_to_regions(filename, solar_data, wind_data, wind_regions, solar_regions):
    jan_march_wind=0
    apr_sep_wind=0
    oct_dec_wind=0
    jan_march_solar=0
    apr_sep_solar=0
    oct_dec_solar = 0

    
    with open(filename) as file:
            reader = csv.reader(file)
            next(reader)
            while True:
                try:
                    next_line = next(reader)
                except StopIteration:
                    break
                if (next_line[0] == 'WIND-C') or (next_line[0] == 'WIND'):
                    jan_march_wind+= float(next_line[2])
                    apr_sep_wind+= float(next_line[3])
                    oct_dec_wind+= float(next_line[4])
                    wind_regions[next_line[1]] += float(next_line[3])
                if next_line[0] == 'SOLAR':
                    jan_march_solar+= float(next_line[2])
                    apr_sep_solar+= float(next_line[3])
                    oct_dec_solar+= float(next_line[4])
                    solar_regions[next_line[1]] += float(next_line[3])

    for i in wind_regions:
        wind_regions[i] = float(wind_regions[i]/apr_sep_wind)
    for i in solar_regions:
        solar_regions[i] = float(solar_regions[i]/apr_sep_solar)
    
    for i in wind_data:
        wind_regions_copy = dict(wind_regions)
        for j in wind_regions_copy:
            wind_regions_copy[j] = float(wind_regions_copy[j]*wind_data[i])
        wind_data[i] = wind_regions_copy
            
    for i in solar_data:
        solar_regions_copy = dict(solar_regions)
        for j in solar_regions_copy:
            solar_regions_copy[j] = float(solar_regions_copy[j]*solar_data[i])
        solar_data[i] = solar_regions_copy
        

In [6]:
def write_to_csv(wind_data, solar_data):
    with open('wind_per_15min.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
                                quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['Date','Time','Coast', 'East', 'FarWest', 'North', 'NorthCentral', 'Northwest', 'South', 
                    'SouthCentral','West'])
        count = 4
        for i in wind_data:
            if count == 4:
                count = 0
                split_string = i.split()
                split_string[1] = split_string[1][:2]
                temp = split_string
                for j in wind_data[i]:
                    temp.append(str(wind_data[i][j]))
                writer.writerow(temp)
            count+=1

    
    with open('solar_per_15min.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
                                quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['Time','Coast', 'East', 'FarWest', 'North', 'NorthCentral', 'Northwest', 'South', 
                    'SouthCentral','West'])
        count = 4
        for i in solar_data:
            if count == 4:
                count = 0
                split_string = i.split()
                split_string[1] = split_string[1][:2]
                temp = split_string
                for j in solar_data[i]:
                    temp.append(str(solar_data[i][j]))
                writer.writerow(temp)
            count+=1

In [8]:
data = load_file_60d('60d_SCED_SMNE_GEN_RES-17-APR-18.csv', 'may_generation.csv')
code_to_type('60d_SCED_Gen_Resource_Data-17-APR-18.csv', data)

wind = {}
solar = {}
percent_wind = {}
percent_solar = {}
wind_solar_added('may_generation.csv',data, wind, solar, percent_wind, percent_solar)


wind_regions = {'Coast':0, 'East':0, 'FarWest':0, 'North':0, 'NorthCentral':0, 'Northwest':0, 'South':0, 
                    'SouthCentral':0,'West':0}
solar_regions= {'Coast':0, 'East':0, 'FarWest':0, 'North':0, 'NorthCentral':0, 'Northwest':0, 'South':0, 
                    'SouthCentral':0,'West':0}
map_to_regions('zonal_generator_capacities_wo_biomass.csv',solar, wind, wind_regions, solar_regions)

write_to_csv(wind, solar)
